In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
# Data
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_validation_samples, tf.int64)

def scale(image, label) :
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)
    
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)
    
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [3]:
#Model
input_size = 784
output_size = 10
hidden_layer_size = 500

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(output_size, activation = 'softmax')   
])

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
#Training
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/5
540/540 - 19s - loss: 0.2323 - accuracy: 0.9299 - val_loss: 0.1256 - val_accuracy: 0.9645 - 19s/epoch - 35ms/step
Epoch 2/5
540/540 - 14s - loss: 0.1051 - accuracy: 0.9684 - val_loss: 0.0871 - val_accuracy: 0.9748 - 14s/epoch - 27ms/step
Epoch 3/5
540/540 - 14s - loss: 0.0791 - accuracy: 0.9773 - val_loss: 0.0514 - val_accuracy: 0.9862 - 14s/epoch - 27ms/step
Epoch 4/5
540/540 - 14s - loss: 0.0559 - accuracy: 0.9835 - val_loss: 0.0517 - val_accuracy: 0.9860 - 14s/epoch - 26ms/step
Epoch 5/5
540/540 - 14s - loss: 0.0487 - accuracy: 0.9856 - val_loss: 0.0412 - val_accuracy: 0.9877 - 14s/epoch - 27ms/step


In [6]:
test_loss, test_accuracy = model.evaluate(test_data)

2/2 [==============================] - 1s 316ms/step - loss: 0.0820 - accuracy: 0.9789


In [7]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.89%
